# Домашнее задание 4

## Реализация градиентного бустинга

В рамках этой задачи нужно написать градиентный бустинг над решающими деревьями в задаче классификации. В качестве функции потерь предлагается взять **log loss**. Про рего можно прочитать подробнее здесь: https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss


$y_i$ это правильный ответ (0 или 1), $\hat{y}_i$ это ваше предскзаание

Может показаться, что надо максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$, где $y_i$

Да, но нет. Лучше максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(f(\hat{y}_i)) + (1 - y_i) \log(1 - f(\hat{y}_i))$, где $f(x) = \frac{1}{1 + e^{-x}}$. Благодаря этому у вас не будет ограничений на принимаеммые значения для $\hat{y}_i$

### Задание 1

Напишите вычисление производной f(x), обычно её называют **сигмоида**.

In [4]:
import numpy as np

In [5]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def der_sigmoid(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

In [6]:
der_sigmoid(0) == 0.25

True

In [7]:
der_sigmoid(np.array([0, 0])) == np.array([0.25, 0.25])

array([ True,  True])

In [8]:
der_sigmoid(np.log(3)) == 0.1875

True

**Значение для формы:**

In [9]:
print(round(der_sigmoid(np.array([-10, 4.1, -1, 2])).sum() + sigmoid(0.42), 4))

0.9212


Хорошо, теперь мы умеем считать производную функции f, но надо найти производную log loss-а по $\hat{y}$ в первом варианте 

$$y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$$

### Задание 2

Напишите вычисление производной log loss-a

In [26]:
def der_log_loss(y_hat, y_true):
    """
    0 < y_hat < 1
    """
    assert np.all(y_hat < 1) and np.all(y_hat > 0)
    
    return (y_true - y_hat) / (y_hat - y_hat**2)

In [27]:
der_log_loss(0.5, 0) == -2

True

In [30]:
der_log_loss(0.5, 1) == 2

True

In [31]:
np.round(der_log_loss(np.array([0.8, 0.8]), np.array([1, 1])), 2) == np.array([1.25, 1.25])

array([ True,  True])

**Значение для формы**

In [32]:
print(round(-sum(der_log_loss((x + 1) / 100., x % 2) for x in range(99)), 2))

69.82


Отлично теперь мы можем воспользоваться производной сложной функции и получить вычисление градиента формулы по второму варианту:

$$L(\hat{y}, y) = \sum_{i=1}^n y_i \log(f(\hat{y}_i)) + (1 - y_i) \log(1 - f(\hat{y}_i))$$ $$f(x) = \frac{1}{1 + e^{-x}}$$

In [33]:
def calc_gradient(y_hat, y_true):
    return der_log_loss(sigmoid(y_hat), y_true) * der_sigmoid(y_hat)

Теперь мы можем написать код градиентного бустинга для классификации

### Задание 3

Допишите класс

In [41]:
from sklearn.base import BaseEstimator # чтобы поддержать интерфейс sklearn
from sklearn.tree import DecisionTreeRegressor # для обучения на каждой итерации

In [120]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        """
        tree_params_dict - словарь параметров, которые надо использовать при обучении дерева на итерации
        iters - количество итераций
        tau - коэффициент перед предсказаниями деревьев на каждой итерации
        """
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        
        self.estimators = []
        curr_pred = 0
        
        for iter_num in range(self.iters):
            
            # Нужно найти градиент функции потерь по предсказниям в точке curr_pred
            grad = calc_gradient(curr_pred, y_data)
            
            # Мы максимизируем, поэтому надо обучить DecisionTreeRegressor с параметрами 
            # tree_params_dict по X_data предсказывать grad
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, grad)
            
            self.estimators.append(algo)
            
            # все предсказания домножаются на tau и обновляется переменная curr_pred
            curr_pred += self.tau * algo.predict(X_data)
            
            
        #return self.estimators
       
    
    def predict(self, X_data):
        
        # изначально все предскзания нули
        res = np.zeros(X_data.shape[0])
        
        for estimator in self.estimators:
            # нужно сложить все предсказания деревьев с весом self.tau
            res += self.tau * estimator.predict(X_data)
            
        return (res > 0).astype(int)
       

## Проверка качества полученного класса (в самом низу код для формы)

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

In [121]:
# для оценки качества
from sklearn.model_selection import cross_val_score

# для генерации датасетов
from sklearn.datasets import make_classification

# для сравнения
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [122]:
X_data, y_data = make_classification(n_samples=1000, n_features=10, random_state=42)

In [144]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=1000,
    tau = 0.01
)

In [145]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='accuracy'))

0.922

In [125]:
np.mean(cross_val_score(DecisionTreeClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.8540288007200181

In [126]:
np.mean(cross_val_score(XGBClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.8969841246031149

In [127]:
np.mean(cross_val_score(LogisticRegression(), X_data, y_data, cv=5, scoring='accuracy'))

0.8560087002175054

**Значение для формы**

In [143]:
from sklearn.model_selection import StratifiedKFold

print(round(np.mean(cross_val_score(SimpleGB(
    tree_params_dict={
        'max_depth': 4
    },
    iters=1000,
    tau = 0.01
), X_data, y_data, cv=StratifiedKFold(4, random_state=42), scoring='accuracy')), 3))

0.923
